### Import Libraries

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Import Useful Functions

In [2]:
parent_directory = os.path.join(os.environ["BLACK_BOX"])
evals_path = os.path.join(parent_directory, "experiments/results/evaluation_statistics")
sys.path.append(parent_directory)

from experiments.utils.validation_utils import load_eval_from_csv

### Get Trained Agent Model Paths

In [3]:
# evaluation RL model folders located in ./experiments/results/evaluation_statistics folder
folder_name_1 = "evaluation_PPOTrainer_highway_environment:highway-environment-v0_2022-08-07_21-22-18mdgw9lf4_Agent0"
folder_name_2 = "evaluation_PPOTrainer_highway_environment:highway-environment-v0_2022-08-19_19-34-16_a2cqydg_Agent1"
folder_name_3 = "evaluation_PPOTrainer_highway_environment:highway-environment-v0_2022-08-19_23-31-04_y6fkw_8_Agent2"
folder_name_4 = "evaluation_PPOTrainer_highway_environment:highway-environment-v0_2022-08-20_09-14-16_3_zp7wz_Agent3"
folder_name_5 = "evaluation_PPOTrainer_highway_environment:highway-environment-v0_2022-08-08_09-45-04y_mw9iyt_Agent0123"

### Get All Files In the Directory

In [4]:
eval_directory_1 = os.path.join(evals_path, folder_name_1)
eval_directory_2 = os.path.join(evals_path, folder_name_2)
eval_directory_3 = os.path.join(evals_path, folder_name_3)
eval_directory_4 = os.path.join(evals_path, folder_name_4)
eval_directory_5 = os.path.join(evals_path, folder_name_5)

episode_evals_results_1 = [eval_directory_1 + "/" + f for f in os.listdir(eval_directory_1) if os.path.isfile(os.path.join(eval_directory_1, f))]
episode_evals_results_2 = [eval_directory_2 + "/" + f for f in os.listdir(eval_directory_2) if os.path.isfile(os.path.join(eval_directory_2, f))]
episode_evals_results_3 = [eval_directory_3 + "/" + f for f in os.listdir(eval_directory_3) if os.path.isfile(os.path.join(eval_directory_3, f))]
episode_evals_results_4 = [eval_directory_4 + "/" + f for f in os.listdir(eval_directory_4) if os.path.isfile(os.path.join(eval_directory_4, f))]
episode_evals_results_5 = [eval_directory_5 + "/" + f for f in os.listdir(eval_directory_5) if os.path.isfile(os.path.join(eval_directory_5, f))]

### Return All Statistics for Each Episode

In [5]:
def get_stats(episode_evals_results):
    full_eps_results = []
    eps_rewards = []
    n_collisions = []
    n_impossibles = []

    for eps_stat in episode_evals_results:

        stat_df = load_eval_from_csv(file_name=eps_stat)
        full_eps_results.append(stat_df)

        eps_avg_reward = stat_df["ego_rewards"].mean(skipna = True)
        eps_sum_reward = stat_df["ego_rewards"].sum(skipna = True)
        eps_rewards.append(eps_sum_reward)

        is_collision = stat_df["is_collision"].iloc[0]
        n_collisions.append(is_collision)

        is_impossible = stat_df["is_impossible"].iloc[0]
        n_impossibles.append(is_impossible)
    
    return full_eps_results, eps_rewards, n_collisions, n_impossibles

In [6]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), "full") / w

In [ ]:
full_eps_results_1, eps_rewards_1, n_collisions_1, n_impossibles_1 = get_stats(episode_evals_results_1)
full_eps_results_2, eps_rewards_2, n_collisions_2, n_impossibles_2 = get_stats(episode_evals_results_2)
full_eps_results_3, eps_rewards_3, n_collisions_3, n_impossibles_3 = get_stats(episode_evals_results_3)
full_eps_results_4, eps_rewards_4, n_collisions_4, n_impossibles_4 = get_stats(episode_evals_results_4)
full_eps_results_5, eps_rewards_5, n_collisions_5, n_impossibles_5 = get_stats(episode_evals_results_5)

### Plot Results

In [9]:
y_values_1 = eps_rewards_1
y_values_2 = eps_rewards_2
y_values_3 = eps_rewards_3
y_values_4 = eps_rewards_4
y_values_5 = eps_rewards_5

x_values = list(range(1, len(y_values_1) + 1, 1))

In [ ]:
plt.title("Evaluation Episode Reward")

plt.plot(x_values, y_values_1, color="red")
plt.plot(x_values, y_values_2, color="green")
plt.plot(x_values, y_values_3, color="blue")
plt.plot(x_values, y_values_4, color="yellow")
plt.plot(x_values, y_values_5, color="black")

plt.show()

In [ ]:
y_moving_avg_1 = moving_average(np.array(y_values_1), 20)
y_moving_avg_2 = moving_average(np.array(y_values_2), 20)
y_moving_avg_3 = moving_average(np.array(y_values_3), 20)
y_moving_avg_4 = moving_average(np.array(y_values_4), 20)
y_moving_avg_5 = moving_average(np.array(y_values_5), 20)

x_moving_values = list(range(1, len(y_moving_avg_1) + 1, 1))

plt.title("Evaluation Moving Average Episode Reward")
plt.plot(x_moving_values, y_moving_avg_1, color="red", label="Agent-0")
plt.plot(x_moving_values, y_moving_avg_2, color="green", label="Agent-1")
plt.plot(x_moving_values, y_moving_avg_3, color="blue", label="Agent-2")
plt.plot(x_moving_values, y_moving_avg_4, color="yellow", label="Agent-3")
plt.plot(x_moving_values, y_moving_avg_5, color="black", label="Agent-Vanilla")

text = "Agent-0 Collisions : " + str(n_collisions_1.count(True)) \
        + "\n\nAgent-1 Collisions : " + str(n_collisions_2.count(True)) \
        + "\n\nAgent-2 Collisions : " + str(n_collisions_3.count(True)) \
        + "\n\nAgent-3 Collisions : " + str(n_collisions_4.count(True)) \
        + "\n\nAgent-Vanilla Collisions : " + str(n_collisions_5.count(True))

plt.text(1100, 28, text)

plt.xlabel("Number of Scenarios")
plt.ylabel("Running Average Episode Reward")

plt.legend()

save_name = "running_reward"
plt.savefig(save_name + ".jpg", dpi=600, bbox_inches="tight")

plt.show()